In [4]:
# bibliotecas
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
# criar um navegador
nave = webdriver.Chrome()

# importar/visualizar base de dados

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


Definições das funções de busca no google e no buscapé

In [5]:
def busca_google_shopping(nave, produto, termos_banidos, preco_minimo, preco_maximo):
    # entrar no google
    nave.get('https://www.google.com.br/')

    # tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    
    
    # presquisar o produto
    nave.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    nave.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    
    # clicar na aba Shopping
    elementos = nave.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # pegar a lista de resultado da busca no google shopping
    lista_resultados = nave.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

    # para cada resultado, ele vai verificar se o resultado corresponde as nossas condições
    lista_ofertas = [] # lista que a função vai me da como resposta
    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
        nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome = nome.lower()

        # verificaçao do nome - se no nome tem algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
                
        # verificar se no nome tem todos os termos do nome do produto
        tem_todos_termos_produtos = True 
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produtos = False  

        if not tem_termos_banidos and tem_todos_termos_produtos: # verificando o nome
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                # verificando se o preço ta dentro do minimo e maximo          
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link =  resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By. XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
                
                
    return lista_ofertas


def busca_buscape(nave, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    # entrar no buscape
    nave.get("https://www.buscape.com.br/")
    
    # pesquisar pelo produto no buscapé
    nave.find_element(By.CLASS_NAME, 'AutoCompleteStyle_textBox__eLv3V').send_keys(produto, Keys.ENTER)   
    
    # pegar a lista de resultado da busca no buscape
    time.sleep(5)
    lista_resultados = nave.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')  
    
    # para cada resultado
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__JXsj_').text
            nome = resultado.get_attribute('title')
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            # verificaçao do nome - se no nome tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True
                    
            # verificar se no nome tem todos os termos do nome do produto
            tem_todos_termos_produtos = True 
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produtos = False  
                    
            if not tem_termos_banidos and tem_todos_termos_produtos:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))            
        except:
            pass
        
    return lista_ofertas

Construção da nossa lista de ofertas encontradas

In [6]:
tabela_ofertas = pd.DataFrame()
             
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
                   
    lista_ofertas_google_shopping = busca_google_shopping(nave, produto, termos_banidos, preco_minimo, preco_maximo)    
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preço', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nave, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preço', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None
display(tabela_ofertas)

C:\Users\wvans\AppData\Local\Temp\ipykernel_14272\1241351190.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\wvans\AppData\Local\Temp\ipykernel_14272\1241351190.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_buscape)
C:\Users\wvans\AppData\Local\Temp\ipykernel_14272\1241351190.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\wvans\AppData\Local\Temp\ipykernel_14272\1241351190.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_

,produto,preço,link
0,apple iphone 12 4gb/64gb 6.1 azul,3456.99,https://www.google.com.br/url?url=https://www....
1,apple iphone 12 4gb/64gb 6.1 branco,3434.99,https://www.google.com.br/url?url=https://www....
2,"iphone 11 apple (64gb) lilás tela 6,1"" câmera ...",3407.10,https://www.google.com.br/url?url=https://www....
3,"iphone 11, preto, 64gb, tela hd 6,1, câm. 12mp...",3099.00,https://www.google.com.br/url?url=https://loja...
0,smartphone apple iphone se 3 vermelho 64gb 12....,3099.00,https://www.buscape.com.br/celular/smartphone-...
1,smartphone apple iphone se 2 vermelho 64gb 12....,3249.00,https://www.buscape.com.br/celular/smartphone-...
0,placa de vídeo geforce rtx 3060 v2 oc 12gb gdd...,4299.90,https://www.google.com.br/url?url=https://www....
1,placa de video msi geforce rtx 3060 ti gaming ...,4180.90,https://www.google.com.br/url?url=https://www....
2,placa de vídeo evga nvidia geforce rtx 3060 ti...,4049.91,https://www.google.com.br/url?url=https://www....
3,placa de video asus geforce rtx 3060 ti dual o...,4443.33,https://www.google.com.br/url?url=https://www....


Exportar a base de ofertas para o excel

In [7]:
# exportar pro excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

Enviando o e-mail

In [8]:
# enviar para o email o resultado da tabela
import win32com.client as win32

# verificando se existe alguma ofertar na tabela de ofertas
if len(tabela_ofertas) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application') 
    mail = outlook.CreateItem(0)
    mail.To = 'contatestepython1@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f'''
  <p>Prezados</p>
  <p>Encontramos alguns produtos em ofertas dentro da faixa de preço desejada, segue com a tabela para mais detalhes</p>
   {tabela_ofertas.to_html(index=False)}
  <p>Qualquer dúvida estou a disposição</p>
  <p>att., Weberson</p>    
  '''
    mail.Send()
    
nave.quit()    